In [8]:
import pandas as pd
import geocoder

from tqdm import tqdm

import uuid
from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocessing import Pool
import pickle

In [2]:
PATH_DATA_POLLING = "../../data/polling/Updated Statewide Polling Locations 10132020.xlsx"

In [3]:
df_polling = pd.read_excel(PATH_DATA_POLLING)
df_polling["address_complete"] = df_polling["Street Number"] + " " + df_polling["Street Name"].str.title() + ", " + df_polling["City"].str.title() + ", " + df_polling["State"] + " " + df_polling["Zip5"].astype(str)

In [4]:
addresses = list(df_polling.address_complete)

In [5]:
def geocode_backoff(address, verbose=False):
    isok = False
    while not isok:
        address = ", ".join(address.split(", ")[:-1])
        if address == "":
            isok = True
        else:
            response = geocoder.osm(address)
            isok = response.ok
            if not isok and verbose:
                print("backing off")
    return response

In [68]:
for address in addresses[:30]:
    print(address)
    print(geocode_backoff(address, True).latlng)
    print("---------------------------------")

245 Industrial Dr, Baxley, GA 31513
[31.782022, -82.342977]
---------------------------------
3397 Hatch Pky N, Baxley, GA 31513
[31.7902269, -82.3517574]
---------------------------------
392 Altamaha School Rd, Baxley, GA 31513
backing off
[31.86896, -82.393356]
---------------------------------
353 Blackshear Hwy, Baxley, GA 31513
[31.732963, -82.345131]
---------------------------------
252 W. Parker St., Baxley, GA 31513
[31.7813196, -82.3573872]
---------------------------------
5274 Lake Mayers Rd., Baxley, GA 31513
backing off
[31.75829762905059, -82.47729074419406]
---------------------------------
83 S Oak St, Baxley, GA 31513
[31.778087, -82.344803]
---------------------------------
525 Zion Baptist Ch. Rd., Baxley, GA 31513
backing off
[31.859586, -82.183802]
---------------------------------
16 Pratt Hutto Rd Se, Surrency, GA 31563
backing off
backing off
None
---------------------------------
636 Austin Avenue E, Pearson, GA 31642
[31.291522758027735, -82.84048759159688]


In [ ]:
list_latlon = []
for address in tqdm(addresses):
    list_latlon.append(geocode_backoff(address))
with open("../../data/latlon_polling", "wb") as f:
    pickle.dump(list_latlon, f, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
df_polling["lat"] = [latlon[0] if latlon is not None else None for latlon in list_latlon]
df_polling["lon"] = [latlon[1] if latlon is not None else None for latlon in list_latlon]

In [25]:
df_polling.to_csv("../../data/polling_geocoded.csv", index=False)

In [ ]:
# with Pool(5) as p:
#     list_latlon_pool = list(tqdm(p.imap(geocode_address, addresses[:num_to_geocode]), total=num_to_geocode))
    
# list_latlon_runner = []
# def runner():
#     threads = []
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         for i in range(num_to_geocode):
#             file_name = uuid.uuid1()
#             address = addresses[i]
#             threads.append(executor.submit(geocode_address, address))
#         for task in tqdm(as_completed(threads), total=num_to_geocode):
#             list_latlon_runner.append(task.result())

# runner()